In [1]:
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

Using device cuda:0


## 读取dataset

In [2]:
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

## 定义模型

In [3]:
from models.CVAE import CVAE

model = CVAE(sample_size = 784, 
                  condition_size = 10, 
                  encoder_layer_sizes = [512,512], 
                  latent_size = 20, 
                  decoder_layer_sizes = [512,512]).to(device)
print(model)

CVAE(
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Linear(in_features=794, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
    )
    (linear_means): Linear(in_features=512, out_features=20, bias=True)
    (linear_log_var): Linear(in_features=512, out_features=20, bias=True)
  )
  (decoder): Decoder(
    (sequential): Sequential(
      (0): Linear(in_features=30, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=784, bias=True)
    )
  )
)


## loss function, optimizer

In [4]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

optimizer = optim.Adam(model.parameters(), lr=1e-3)

## 训练

In [5]:
from utils.mnist import idx2onehot

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (sample, condition) in enumerate(train_loader):        
        sample, condition = sample.to(device), condition.to(device)
        sample = sample.view(-1, 784)
        condition = idx2onehot(condition, 10)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(sample, condition)
        recon_batch = torch.sigmoid(recon_batch)
        loss = loss_fn(recon_batch, sample, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(sample), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(sample)))

    print('====> Epoch: {} Average loss: {:.7f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

def test_recon(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (sample, condition) in enumerate(test_loader):
            sample, condition = sample.to(device), condition.to(device)
            sample = sample.view(-1, 784)
            condition = idx2onehot(condition, 10)
            recon_batch, mu, logvar = model(sample, condition)
            recon_batch = torch.sigmoid(recon_batch)
            test_loss += loss_fn(recon_batch, sample, mu, logvar).item()
            if i == 0:
                n = min(sample.size(0), 8)
                comparison = torch.cat([sample.view(batch_size, 1, 28, 28)[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'mnist_results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    
epoch = 0

In [25]:
for epoch in range(epoch, epoch + 10):
    log_interval = 100
    train(epoch)
    test_recon(epoch)
    with torch.no_grad():
        z = torch.randn(64, 20).to(device)
        c = torch.randint(0, 10, (64,)).to(device)
        print(c.view(8,8))
        c = idx2onehot(c, 10)
        sample = model.decode(torch.cat((z, c), dim=-1)).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'mnist_results/sample_' + str(epoch) + '.png')

Train Epoch: 9 [0/60000 (0%)]	Loss: 91.872421
Train Epoch: 9 [12800/60000 (21%)]	Loss: 93.913864
Train Epoch: 9 [25600/60000 (43%)]	Loss: 93.474792
Train Epoch: 9 [38400/60000 (64%)]	Loss: 96.174927
Train Epoch: 9 [51200/60000 (85%)]	Loss: 94.103897
====> Epoch: 9 Average loss: 94.2032882
====> Test set loss: 95.3874
tensor([[2, 9, 2, 0, 8, 4, 0, 5],
        [2, 3, 7, 4, 7, 4, 6, 6],
        [8, 9, 5, 6, 3, 2, 0, 3],
        [1, 4, 4, 0, 5, 7, 8, 3],
        [4, 0, 2, 6, 4, 5, 6, 5],
        [2, 3, 2, 6, 3, 1, 8, 0],
        [5, 0, 4, 9, 4, 1, 1, 7],
        [9, 2, 2, 1, 0, 5, 5, 6]], device='cuda:0')
Train Epoch: 10 [0/60000 (0%)]	Loss: 95.769569
Train Epoch: 10 [12800/60000 (21%)]	Loss: 90.567062
Train Epoch: 10 [25600/60000 (43%)]	Loss: 92.921761
Train Epoch: 10 [38400/60000 (64%)]	Loss: 100.566666
Train Epoch: 10 [51200/60000 (85%)]	Loss: 94.633575
====> Epoch: 10 Average loss: 93.9913743
====> Test set loss: 95.3167
tensor([[6, 3, 5, 8, 5, 2, 6, 0],
        [2, 6, 8, 4, 7, 1, 4, 6

## 可视化结果

In [26]:
for i in range(0, 10):
    z = torch.randn(64, 20).to(device)
    c = torch.tensor(i).expand(64).to(device)
    c = idx2onehot(c, 10)
    with torch.no_grad():
        sample = model.decode(torch.cat((z, c), dim=-1)).cpu()
        save_image(sample.view(64, 1, 28, 28),
                       'mnist_results/condition_' + str(i) + '.png')